In [7]:
# File: interactive_paint_with_dynamic_brush.py
import cv2
import mediapipe as mp
import numpy as np
import math

# Inicialización de Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Configuración de colores
colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0), (0, 255, 255)]  # Rojo, Verde, Azul, Amarillo
current_color = colors[0]
brush_size = 10

# Variables del menú y pintura
menu_width = 100
drawing = False
canvas = None  # Para almacenar lo dibujado
dominant_hand = None  # Será "Left" o "Right"

# Funciones auxiliares
def draw_menu(frame):
    """Dibuja los colores del menú sin fondo."""
    for i, color in enumerate(colors):
        cx, cy = 50, 100 + i * 100
        cv2.circle(frame, (cx, cy), 30, color, -1)

def detect_dominant_hand(results):
    """Detecta si la mano es izquierda o derecha."""
    if results.multi_handedness:
        for hand in results.multi_handedness:
            return hand.classification[0].label  # "Left" o "Right"
    return None

def count_visible_fingers(hand_landmarks):
    """Cuenta el número de dedos visibles."""
    finger_tips = [
        mp_hands.HandLandmark.THUMB_TIP,
        mp_hands.HandLandmark.INDEX_FINGER_TIP,
        mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
        mp_hands.HandLandmark.RING_FINGER_TIP,
        mp_hands.HandLandmark.PINKY_TIP
    ]

    visible_fingers = 0
    for tip in finger_tips:
        tip_coord = hand_landmarks.landmark[tip]
        base_coord = hand_landmarks.landmark[tip - 2]  # Base de cada dedo
        if tip_coord.y < base_coord.y:  # Dedo levantado si la punta está por encima de la base
            visible_fingers += 1

    return visible_fingers

def calculate_brush_size(hand_landmarks):
    """Calcula el tamaño de la brocha según la distancia entre los dedos índice y medio."""
    index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
    middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]

    # Distancia euclidiana entre los dos dedos
    distance = math.sqrt((index_tip.x - middle_tip.x)**2 + (index_tip.y - middle_tip.y)**2)
    return int(distance * 300)  # Escalar la distancia para que sea visible

# Iniciar cámara
cap = cv2.VideoCapture(0)
calibrating = True  # Modo calibración

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if canvas is None:
        canvas = np.zeros_like(frame)  # Inicializar el lienzo

    frame = cv2.flip(frame, 1)  # Flip horizontal para parecer espejo
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    frame_height, frame_width, _ = frame.shape

    if calibrating:
        # Mostrar mensaje para calibrar
        cv2.putText(frame, "Muestra tu mano dominante", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Detectar la mano dominante
        if results.multi_hand_landmarks:
            dominant_hand = detect_dominant_hand(results)
            if dominant_hand:
                calibrating = False  # Salir de calibración

    else:
        # Detección de manos (solo rastrear la dominante)
        if results.multi_handedness and results.multi_hand_landmarks:
            for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
                handedness = results.multi_handedness[i].classification[0].label
                if handedness != dominant_hand:
                    continue  # Ignorar manos que no sean la dominante

                # Coordenadas del índice
                index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                x, y = int(index_finger.x * frame_width), int(index_finger.y * frame_height)

                # Contar dedos visibles
                visible_fingers = count_visible_fingers(hand_landmarks)

                if visible_fingers == 0:
                    # Puño cerrado, desactivar dibujo
                    drawing = False
                else:
                    # Ajustar el tamaño del pincel dinámicamente
                    drawing = True
                    if visible_fingers == 1:
                        brush_size = 10  # Tamaño mínimo para un dedo
                    elif visible_fingers >= 2:
                        brush_size = calculate_brush_size(hand_landmarks)

                # Detectar interacción con el menú
                wrist_x = int(hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x * frame_width)
                if wrist_x < menu_width:
                    # Cambiar color si se toca un círculo de color
                    for i, color in enumerate(colors):
                        cx, cy = 50, 100 + i * 100
                        if math.hypot(x - cx, y - cy) < 30:
                            current_color = color
                else:
                    # Dibujar según la posición del índice
                    if drawing:
                        cv2.circle(canvas, (x, y), brush_size, current_color, -1)  # Pintar

                # Mostrar el color del índice
                cv2.circle(frame, (x, y), 10, current_color, -1)  # Colorear la punta del índice

    # Combinar el lienzo con el frame
    frame = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)

    # Dibujar el menú
    draw_menu(frame)

    # Mostrar el frame
    cv2.imshow("Paint Interactivo", frame)

    key = cv2.waitKey(1)
    if key & 0xFF == 27:  # Presionar ESC para salir
        break
    elif key & 0xFF == ord('d'):  # Activar/desactivar dibujo
        drawing = not drawing

cap.release()
cv2.destroyAllWindows()
